In [ ]:
# %pip install

In [2]:
import csv
import requests
import time
import time
import pandas as pd
# from dotenv import load_dotenv
# import os
from googlesearch import search
import re
import json
import nltk
from nltk.corpus import stopwords

#### FETCH LEGITIMATE URLS DATA

In [33]:
def fetch_from_coingecko(url):
    page = 1
    all_exchanges = []

    while True:
        # Add the page parameter to the request
        params = {"per_page": 300, "page": page}

        try:
            # Make the request
            response = requests.get(url, params=params)

            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                # If no data is returned, stop the loop
                if not data:
                    break
                # Add the fetched data to the list
                all_exchanges.extend(data)
                print(f"Fetched page {page} with {len(data)} exchanges.")
                # Move to the next page
                page += 1
                # Add a delay to avoid hitting the rate limit
                time.sleep(1)

            elif response.status_code == 429:
                # Handle rate limit error
                print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait 60 seconds before retrying

            else:
                # Handle other errors
                print(f"Failed to fetch data. Status code: {response.status_code}. Reason: {response.reason}")
                break

        except requests.exceptions.RequestException as e:
            # Handle network errors
            print(f"Network error: {e}")
            break

    return all_exchanges

# Function to fetch market data (coins list)
def fetch_coin_market_data(url):
    page = 1
    all_coins = []


    while True:
      params = {'vs_currency': 'usd', 'order': 'market_cap_desc', 'per_page': 250, 'page': page, 'x_cg_demo_api_key': "CG-Y7Gt7HP5cmaK74WbKKbhh7dZ"}

      try:
          response = requests.get(url, params=params)

          if response.status_code == 200:
                data = response.json()
                # If no data is returned, stop the loop
                if not data:
                    break
                # Add the fetched data to the list
                all_coins.extend(data)
                print(f"Fetched page {page} with {len(data)} exchanges.")
                # Move to the next page
                page += 1
                # Add a delay to avoid hitting the rate limit
                time.sleep(1)

          elif response.status_code == 429:
                # Handle rate limit error
                print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait 60 seconds before retrying

          else:
                # Handle other errors
                print(f"Failed to fetch data. Status code: {response.status_code}. Reason: {response.reason}")
                break

      except requests.exceptions.RequestException as e:
            # Handle network errors
            print(f"Network error: {e}")
            break

    return all_coins


# Function to fetch coin details by coin ID
def fetch_coin_details(coin_id):
    while True:
          details_url = f'https://api.coingecko.com/api/v3/coins/{coin_id}'
          details_response = requests.get(details_url, headers={'x_cg_demo_api_key': "CG-Y7Gt7HP5cmaK74WbKKbhh7dZ"})

          if details_response.status_code == 200:
              coin_details = details_response.json()
              homepage_list = coin_details.get('links', {}).get('homepage', [])
              homepage_url = homepage_list[0] if homepage_list else 'N/A'
              print(f"Fetched details for coin {coin_id}: {homepage_url}")
              # Add a delay to avoid hitting the rate limit
              time.sleep(1)

              return homepage_url

          elif details_response.status_code == 429:
              print("Rate limit exceeded, waiting for 60 seconds before retrying...")
              time.sleep(60)

          else:
              print(f"Failed to fetch details for coin {coin_id}. Status code: {details_response.status_code}")
              return 'N/A'

# Function to fetch exchanges from DeFi Llama
def fetch_from_other_source(url):
    # Make the request
    response = requests.get(url)
    if response.status_code == 200:
        # Return the JSON data
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}. Reason: {response.reason}")

        return []


In [34]:
# # function to convert to dictionary
# def convert_to_dict(items):
#     # Create a list to store the name-url mappings
#     dict_list = []

#     # Process each URL in the list
#     for url in items:
#         # Extract the text before the domain extension
#         name = url.split('.')[0]
#         # Create a dictionary for the name and URL
#         dict_list.append({'name': name, 'url': url})

#     return dict_list

# # function to get the format a domain to URL
# def format_domain(domain):
#     if not domain.startswith(('https://', 'http://', 'www.')):
#         return 'https://' + domain
#     return domain

In [35]:
crypto_data_urls = []

In [36]:
# # fetch centralized exchanges from coingecko
# cex_url = "https://api.coingecko.com/api/v3/exchanges"
# gecko_data = fetch_from_coingecko(cex_url)

# for exchange in gecko_data:
#     # Extract the name from the exchange data
#     url = exchange['url']

#     # Append the name to the list
#     crypto_data_urls.append(url)

# # Print the total number of exchanges fetched
# print(f"Total exchanges fetched: {len(gecko_data)}, {gecko_data[0]}")

In [37]:
# # fetch centralized exchanges from coingecko
# gecko_coin_url = 'https://api.coingecko.com/api/v3/coins/markets'
# gecko_coin_data = fetch_coin_market_data(gecko_coin_url)

# for exchange in gecko_coin_data:
#     # Extract the name from the exchange data
#     url = fetch_coin_details(exchange["id"])
#     # print(url)
#     # Append the name to the list
#     crypto_data_urls.append(url)

# # Print the total number of exchanges fetched
# print(f"Total exchanges fetched: {len(gecko_coin_data)}, {gecko_coin_data[0]}")

In [ ]:
pap_exchanges = fetch_from_other_source("https://api.coinpaprika.com/v1/exchanges")

for exchange in pap_exchanges:
    if exchange['links'] and exchange['links'].get('website'):
        url = exchange['links']['website'][0]
    else:
        url = None

# Print the total number of exchanges fetched
print(f"Total exchanges fetched: {len(pap_exchanges)}, {pap_exchanges[0]}")

In [17]:
pap_listings = fetch_from_other_source("https://api.coinpaprika.com/v1/coins")

# Print the total number of exchanges fetched
print(f"Total exchanges fetched: {len(pap_listings)}, {pap_listings[0]}")

Total exchanges fetched: 73561, {'id': 'btc-bitcoin', 'name': 'Bitcoin', 'symbol': 'BTC', 'rank': 1, 'is_new': False, 'is_active': True, 'type': 'coin'}


In [42]:
# pap_listings_df = pd.DataFrame(pap_listings)
# pap_listings_df.to_json('pap_listings.json', orient='records', lines=False)

# from google.colab import files
# files.download('pap_listings.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
def fetch_listing_details(id):
  listing_url = fetch_from_other_source(f"https://api.coinpaprika.com/v1/coins/{id}")
  if not listing_url:
    return ''
  website_list = listing_url['links'].get('website', [])
  website_url = website_list[0] if website_list else ''

  return website_url

In [ ]:
for listings in pap_listings:
    # Extract the name from the exchange data
    url = fetch_listing_details(listings["id"])
    print(url)
    # Append the name to the list
    # crypto_data_urls.append(url)

In [ ]:
 # fetch every exchange protocol from defi llama
# /chains
llama_url = "https://api.llama.fi/protocols"
llama_data = fetch_from_other_source(llama_url)

for exchange in llama_data:
    # Extract the name from the exchange data
    url = exchange['url']
    # print(exchange)
    # Append the name to the list
    crypto_data_urls.append(url)

# Print the total number of exchanges fetched
print(f"Total exchanges fetched: {len(llama_data)}, {llama_data[0]}")

In [ ]:
# coinmarketcap_data = fetch_from_coin_marketcap()

# for exchange in coinmarketcap_data:
#     # Extract the name from the exchange data
#     name = exchange['name']
#     # Append the name to the list
#     crypto_data_names.append(name)

# # Print the total number of exchanges fetched
# print(f"Total exchanges fetched: {len(coinmarketcap_data)}, {coinmarketcap_data[0]}")

In [ ]:
crypto_data_urls = [item for sublist in crypto_data_urls for item in (sublist if isinstance(sublist, list) else [sublist])]
crypto_data_urls = [url for url in crypto_data_urls if url != '']
crypto_data_urls = set(crypto_data_urls)
crypto_data_urls = list(crypto_data_urls)

In [ ]:
# fetch other legitimate websites asides crypto
# other_websites = "https://raw.githubusercontent.com/seigdev/resources/refs/heads/main/top-websites.csv"

# other_raw = fetch_data_csv(other_websites)

# others_df = pd.DataFrame(other_raw)

# others_df = others_df.drop(columns=0)

# others_df = others_df.rename(columns={1: 'url'})

In [ ]:
# df_raw = pd.read_json("legit-exchanges.json")

# # copy the name and urls columns to a new dataframe
# crypto_df = df_raw[['url']].copy()

# crypto_df["url"]  = crypto_df["url"].apply(format_domain)

# coinmarketcap_df = coinmarketcap_df[["url"]].copy()

# legit_df = pd.concat([crypto_df, coinmarketcap_df], ignore_index=True)

# # assign string labels to legitimate urls
# legit_df.loc[:, 'label'] = 'legit'

# # assign string labels to legitimate urls
# legit_df.loc[:, 'label_no'] = 0

# # select first 100
# legit_df = legit_df[:11000]

# # keep only the first occurrence
# legit_df = legit_df.drop_duplicates(subset="url", keep="first")

# legit_df

#### FETCH SCAM URLS DATA

In [ ]:
# # url to fetch scam urls from eth-phishing-detect
# scam_url = "https://raw.githubusercontent.com/MetaMask/eth-phishing-detect/master/src/config.json"
# scam_ex = fetch_from_other_source(scam_url)

# # fetch the list of blacklist urls
# blacklist = scam_ex["blacklist"]

# # convert the blacklist to a dictionary
# blacklist = convert_to_dict(blacklist)

In [ ]:
# with open("scam-exchanges.json", "w") as file:
#     json.dump(blacklist, file)

# blacklist_raw = pd.read_json("scam-exchanges.json")

# # copy the name and urls columns to a new dataframe
# scam_df = blacklist_raw[['url']].copy()

# scam_df["url"] = scam_df["url"].apply(format_domain)

# # assign string labels to legitimate urls
# scam_df.loc[:, 'label'] = 'scam'

# # assign string labels to legitimate urls
# scam_df.loc[:, 'label_no'] = 1

# # select first 100
# # scam_df = scam_df.sample(n=100000, random_state=42).reset_index(drop=True)
# scam_df = scam_df[:50000]

# scam_df

In [ ]:
# # merge both the legit and scam urls together
# urls_df = pd.concat([legit_df, scam_df], ignore_index=True)

# # shuffle the urls across the dataframe
# urls_df = urls_df.sample(frac=1, random_state=42).reset_index(drop=True)

# urls_df

In [ ]:
# urls_df.to_json('crypto_data.json', orient='records', lines=False)